In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt

import pyodbc

Licenze_FromGY["LastUpd"] = pd.to_datetime(Licenze_FromGY["LastUpd"], format="%Y-%m-%d %H:%M:%S.%fZ", errors="coerce")

Licenze_FromGY["LastUpd"].apply(lambda x: dt.datetime.fromisoformat(x))

dt.datetime.strptime("2017-12-14 22:16:24.037 +02:00", "%Y-%m-%d %H:%M:%S.%f %z")

dt.datetime.fromisoformat("2018-11-20 22:20:53.187")

In [2]:
server = 'WkiBI.database.windows.net'
database = 'WkiBI'
username = 'Raffaele.Sportiello'
password = 'sFG(mR-7]?r<U,£x{<G}g}PsdeF{+W(9tb[QP+w4zf:39q:!#"LA_6'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [7]:
#query = "SELECT * FROM [dbo].[Licenze_FromGY];"
query = "Select  RagioneSociale = Concat(A.Denomination collate database_default, ' [' + A.CodiceSAP + ']'), A.CodiceSAP, CodLicense = Concat(S.Description, ' - ', L.Code), L.Aggregate, L.Bundle, Trial, Value = IIF(Value > 10000, 10000, Value), R.Gruppo From  Licenze_FromGY L Join Services_GY S On L.IdService = S.IdService Join AnagraficaAnna_FromGY A On A.Tenant = L.Tenant left join RaccordoLicenzeGruppo R On R.IdService = L.IdService And R.Code = L.Code Where L.Value != 0 and  L.status = 'Active';"
Licenze_FromGY = pd.read_sql(query, cnxn, parse_dates={"EndDate":"%Y-%m-%d", "LastUpd":"%Y-%m-%d %H:%M:%S.%f %z"})

query = "SELECT [IdService], [Description] FROM [dbo].[Services_GY];"
Services = pd.read_sql(query, cnxn)

In [16]:
Licenze_FromGY["LastUpd"] = pd.to_datetime(Licenze_FromGY["LastUpd"], utc=True).dt.tz_convert("Europe/Athens")
Licenze_FromGY["LastUpdDate"] = Licenze_FromGY["LastUpd"].dt.date
Licenze_FromGY["LastUpdTime"] = Licenze_FromGY["LastUpd"].dt.time

In [30]:
Licenze_FromGY["LastUpd"].dt.normalize()

0        2017-12-14 00:00:00+02:00
1        2017-12-14 00:00:00+02:00
2        2017-12-14 00:00:00+02:00
3        2017-12-14 00:00:00+02:00
4        2017-12-14 00:00:00+02:00
                    ...           
244815   2022-08-28 00:00:00+03:00
244816   2022-08-28 00:00:00+03:00
244817   2022-08-28 00:00:00+03:00
244818   2022-08-28 00:00:00+03:00
244819   2022-08-28 00:00:00+03:00
Name: LastUpd, Length: 244820, dtype: datetime64[ns, Europe/Athens]

In [22]:
Licenze_FromGY = Licenze_FromGY.merge(Services, how="left", on="IdService")

In [8]:
Licenze_FromGY.head(20)

,RagioneSociale,CodiceSAP,CodLicense,Aggregate,Bundle,Trial,Value,Gruppo
0,Studio genya - produzione,None,Variazione attività (mod. AA7) - CustomerCount,TXD,Genya Suite Professional - Uso interno,False,1000,0001
1,Studio genya - produzione,None,Variazione attività (mod. AA9) - CustomerCount,TXD,Genya Suite Professional - Uso interno,False,1000,
2,Studio genya - produzione,None,Bilancio - CustomerCount,B,Genya Suite Professional - Uso interno,False,1000,
3,Studio genya - produzione,None,Contabilità - CustomerCount,C,Genya Suite Professional - Uso interno,False,1000,
4,Studio genya - produzione,None,Consolidato nazionale e mondiale - CustomerCount,TXD,Genya Suite Professional - Uso interno,False,1000,
5,Studio genya - produzione,None,Conservazione sostitutiva - ARKonArchiviazione,CS,Conservazione Sostitutiva,False,512,
6,Studio genya - produzione,None,Conservazione sostitutiva - ARKonConservazione,CS,Conservazione Sostitutiva,False,20,
7,Studio genya - produzione,None,Conservazione sostitutiva - ARKonFirmatari,CS,Conservazione Sostitutiva,False,1,
8,Studio genya - produzione,None,Certificazione Unica - CustomerCount,TXD,Genya Suite Professional - Uso interno,False,1000,
9,Studio genya - produzione,None,Redditi enti non commerciali - CustomerCount,TXD,Genya Suite Professional - Uso interno,False,1000,


In [36]:
Licenze_FromGY["Code"].unique()

array(['CustomerCount', 'CustomerSupport', 'BshLimit', 'InternalUseOnly',
       'ARKonArchiviazione', 'ARKonConservazione', 'ARKonFirmatari',
       'CollabB2B', 'CollabCorrispettivi', 'CollabFatturaSmart',
       'CollabPortal', 'CollabSkill', 'CollabSkillAADAI', 'UserCount',
       'OfficeManagement', 'Telematic', 'EnterpriseB2B',
       'AddCollabCorrispettivi', 'AddCustomerCount',
       'AddCollabFatturaSmart', 'AddUserCount', 'AddCollabSkill',
       'AddCollabSkillAADAI', 'LicenseType', 'AddWdDocShare', 'UploadB2B',
       'FsSme'], dtype=object)

In [23]:
Licenze_FromGY["Value"].replace(2147483647,np.nan, inplace=True)

In [17]:
print(Licenze_FromGY.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244820 entries, 0 to 244819
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype                        
---  ------       --------------   -----                        
 0   Tenant       244820 non-null  int64                        
 1   IdService    244820 non-null  object                       
 2   Code         244820 non-null  object                       
 3   Aggregate    244820 non-null  object                       
 4   Bundle       238661 non-null  object                       
 5   Status       244820 non-null  object                       
 6   Value        244820 non-null  int64                        
 7   EndDate      155305 non-null  datetime64[ns]               
 8   Trial        244820 non-null  bool                         
 9   LastUpd      244575 non-null  datetime64[ns, Europe/Athens]
 10  LastUpdDate  244575 non-null  object                       
 11  LastUpdTime  244575 non-null  object   

In [156]:
Licenze_FromGY.groupby(["Value"], dropna=False).agg({"Tenant":"nunique"}).sort_values(by="Value", ascending=False).head(20)

,Tenant
Value,
100000.0,1
2996.0,1
2000.0,1
1500.0,1
1250.0,1
1220.0,1
1200.0,1
1000.0,178
841.0,1


In [29]:
Licenze_FromGY.groupby(["LastUpdDate","LastUpdTime", "LastUpd"]).size()
#.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Customer Success Management\sintesi1.xlsx")

LastUpdDate  LastUpdTime      LastUpd                         
2000-01-01   01:00:00         2000-01-01 01:00:00+02:00             2
2016-10-25   10:08:56.933000  2016-10-25 10:08:56.933000+03:00     67
2016-10-27   12:40:02.902000  2016-10-27 12:40:02.902000+03:00      3
2016-10-28   12:19:35.579000  2016-10-28 12:19:35.579000+03:00      3
             12:38:43.693000  2016-10-28 12:38:43.693000+03:00      3
                                                                 ... 
2022-08-23   12:55:28.521000  2022-08-23 12:55:28.521000+03:00     43
             21:43:09.677000  2022-08-23 21:43:09.677000+03:00     33
2022-08-28   14:21:18.818000  2022-08-28 14:21:18.818000+03:00    137
2022-08-29   16:27:44.333000  2022-08-29 16:27:44.333000+03:00      8
2022-08-31   16:18:17.075000  2022-08-31 16:18:17.075000+03:00      1
Length: 5326, dtype: int64

In [39]:
Licenze_FromGY.groupby(["IdService", "Description", "Code"]).agg({"Value":"sum", "Tenant":"nunique"})

Value  Tenant
IdService Description                    Code                              
AA7       Variazione attività (mod. AA7) AddCustomerCount       0.0     302
                                         CustomerCount     406911.0    4479
                                         CustomerSupport        0.0    4479
                                         InternalUseOnly        0.0       1
AA9       Variazione attività (mod. AA9) AddCustomerCount       0.0     302
...                                                             ...     ...
USP       Redditi società di persone     AddCustomerCount     330.0     417
                                         CustomerCount     406911.0    4479
                                         CustomerSupport        0.0    4479
                                         InternalUseOnly        0.0      56
                                         UserCount              7.0       1

[132 rows x 2 columns]